In [1]:
import os
os.chdir("../../")

In [2]:
from api.metrics.rent_burdened import RentBurdenedMetrics
from api.utils.testing import create_test_db

My own stuff below

In [3]:
import sqlite3

con = sqlite3.connect("./pipeline/database.db")
cur = con.cursor()

rows = cur.execute("""
SELECT *
FROM rent_burdened_households
LIMIT 5
""").fetchall()

rows

[(35, 'all', 52.7029065195125, 3.84632247404482, 2015, 2019, '2015-2019'),
 (36, 'all', 41.6541886411246, 6.20330138729073, 2015, 2019, '2015-2019'),
 (37, 'all', 56.2945422971736, 9.28861601213422, 2015, 2019, '2015-2019'),
 (38, 'all', 51.4589913593531, 3.50474554886712, 2015, 2019, '2015-2019'),
 (39, 'all', 48.790639029861, 3.83623974396546, 2015, 2019, '2015-2019')]

In [4]:
%%capture
%load_ext sql
%sql sqlite:///pipeline/database.db

In [5]:
%%sql
SELECT *
FROM rent_burdened_households
LIMIT 5

 * sqlite:///pipeline/database.db
Done.


area_number,segment,value,std_error,period_start_year,period_end_year,period
35,all,52.7029065195125,3.84632247404482,2015,2019,2015-2019
36,all,41.6541886411246,6.20330138729073,2015,2019,2015-2019
37,all,56.2945422971736,9.28861601213422,2015,2019,2015-2019
38,all,51.4589913593531,3.50474554886712,2015,2019,2015-2019
39,all,48.790639029861,3.83623974396546,2015,2019,2015-2019


In [6]:
from api.utils.database import rows_to_dicts


class RentBurdenedMetrics:
    """
    Metrics for rent burdened households data.
    """

    def __init__(self, con):
        self.con = con

    def rent_burdened(self, year, segment):
        """
        Returns the percent of households who are rent burdened in the area,
        and the area number.
        Args:
            year (int): period ending year to filter by
            segment (str): population segment to filter by
        """
        query = """
        SELECT
            area_number,
            value / 100.00 AS value
        FROM rent_burdened_households
        WHERE period_end_year == {year}
        AND segment == "{segment}"
        """.format(year=year, segment=segment)
        cur = self.con.cursor()
        cur.execute(query)
        rows = rows_to_dicts(cur, cur.fetchall())
        return rows
        
    def rent_max_burdened(self):
        """
        Returns the highest percentage of rent burdened households in an area,
        its area number, and the period.
        """
        query = """
        SELECT
            max(value)/100 AS value,
            area_number,
            period_end_year
        FROM rent_burdened_households
        """
        cur = self.con.cursor()
        cur.execute(query)
        rows = rows_to_dicts(cur, cur.fetchall())
        return rows
    
    def rent_min_burdened(self):
        """
        Returns the smallest percentage of rent burdened households in an area,
        its area number, and the period end year.
        """
        query = """
        SELECT
            min(value)/100 AS value,
            area_number,
            period_end_year
        FROM rent_burdened_households
        """
        cur = self.con.cursor()
        cur.execute(query)
        rows = rows_to_dicts(cur, cur.fetchall())
        return rows
    
    def rent_average_burden_area(self):
        """
        Returns the average percentage of rent burdened households by area
        across all periods.
        """
        query = """
        SELECT
            avg(value)/100 AS "avg value",
            area_number
        FROM rent_burdened_households
        GROUP BY area_number
        """
        cur = self.con.cursor()
        cur.execute(query)
        rows = rows_to_dicts(cur, cur.fetchall())
        return rows
    


In [17]:
import sys
sys.path.append("../")

#from api.metrics.rent_burdened import RentBurdenedMetrics #comment this section if making changes to the RentBurdenedMetrics class
from api.utils.testing import create_test_db

...

def test_RBU():
    rent_burdened_households_table = [
        {
            "area_number": 1,
            "period_end_year": 2019,
            "segment": "all",
            "value": 52.7029065195125
        },
        {
            "area_number": 1,
            "period_end_year": 2018,
            "segment": "all",
            "value": 41.6541886411246
        },
        {
            "area_number": 2,
            "period_end_year": 2019,
            "segment": "all",
            "value": 56.2945422971736
        },
        {
            "area_number": 3,
            "period_end_year": 2014,
            "segment": "all",
            "value": 51.4589913593531
        },
        {
            "area_number": 3,
            "period_end_year": 2018,
            "segment": "all",
            "value": 48.790639029861
        },
        {
            "area_number": 3,
            "period_end_year": 2017,
            "segment": "all",
            "value": 33.9090007521057
        }
    ]
    con, cur = create_test_db(
        scripts=[
            "./pipeline/load/rent_burdened_households.sql"
        ],
        tables={
            "rent_burdened_households": rent_burdened_households_table
        }
    )

    metric = RentBurdenedMetrics(con)

    assert metric.rent_burdened(year=2019, segment="all") == [
        { "area_number": 1, "value": 0.527029065195125 },
        { "area_number": 2, "value": 0.562945422971736 }
    ], "Should have two results for 2019."

    assert metric.rent_max_burdened() == [
        { 'value': 0.562945422971736, 'area_number': 2, 'period_end_year': 2019}
    ], "Double check the max result and period end year."
    
    assert metric.rent_min_burdened() == [
        { "area_number": 3, "value": 0.339090007521057, "period_end_year": 2017}
    ], "Double check the min result and period end year."

    assert metric.rent_average_burden_area() == [
        {'avg value': 0.47178547580318553, 'area_number': 1}, 
        {'avg value': 0.562945422971736, 'area_number': 2}, 
        {'avg value': 0.4471954371377327, 'area_number': 3}
    ], "Check if the results are correct"
    
    print(metric.rent_burdened(year=2018, segment="all"))

In [18]:
test_RBU()

None
[{'area_number': 1, 'value': 0.416541886411246}, {'area_number': 3, 'value': 0.48790639029861}]


In [12]:
metric.rent_burdened(year=2015, segment="all")

NameError: name 'metric' is not defined